In [1]:
from gurobipy import *
import pandas as pd
import numpy as np
import csv

In [2]:
df = pd.read_excel("../data/焚化廠資料.xlsx")
df

,焚化廠名稱,縣市,年份,其他成本,勞動成本,焚化處理量,設計處理量,設計發電量,實際發電量,二氧化碳當量
0,宜蘭縣利澤垃圾資源回收(焚化)廠,宜蘭縣,2011,3.290100e+05,4.685480e+05,199732.2,600,14700000,107838.70,149799.150
1,屏東縣崁頂垃圾資源回收(焚化)廠,屏東縣,2011,8.815880e+05,1.214211e+06,251974.0,900,22500000,123059.80,188980.500
2,苗栗縣垃圾焚化廠,苗栗縣,2011,5.986160e+05,7.687310e+05,163453.4,500,11800000,93279.79,122590.050
3,桃園市垃圾焚化廠,桃園市,2011,1.674725e+06,2.955169e+06,433400.3,1350,35100000,256889.30,325050.225
4,高雄市仁武垃圾資源回收(焚化)廠,高雄市,2011,1.014536e+06,1.403879e+06,443461.0,1350,36500000,238519.70,332595.750
...,...,...,...,...,...,...,...,...,...,...
235,臺北市政府環境保護局內湖垃圾焚化廠,臺北市,2020,2.833317e+05,2.409203e+06,134966.8,900,6000000,30631.83,101225.100
236,臺北市政府環境保護局木柵垃圾焚化廠,臺北市,2020,2.833317e+05,2.409203e+06,216178.4,1500,13500000,79307.77,162133.800
237,臺北市政府環境保護局北投垃圾焚化廠,臺北市,2020,2.833317e+05,2.409203e+06,356538.0,1800,45000000,179905.00,267403.500
238,臺南市永康垃圾資源回收(焚化)廠,臺南市,2020,1.090514e+06,1.429848e+06,251192.7,900,22500000,149915.30,188394.525


In [3]:
yearly_df = df.groupby('年份')

In [4]:
# Sets
Years = list(range(2011, 2021))
K = list(range(24))  # number of DMUs
I_V = [0,1,2]  # variable input amount
I_F = [0,1]  # fixed input amount
J = [0]  # output 
M = [0]  # bad output
years = list(range(1,11))
dmus = list(range(1,25))

### Create a production data object to store the production data including:

In [5]:
# Object of production data
class prod_data:
    """
    Attr:
    * x[i][r]: Input i of DMU r
    * y[j][r]: Output j of DMU r
    * b[k][r]: Bad output k of DMU r
    """
    def __init__(self, X_Vdata, X_Fdata, Ydata, Bdata):
        self.x_v = X_Vdata
        self.x_f = X_Fdata
        self.y = Ydata
        self.b = Bdata

In [6]:
yearly_prod_data = {}
for year in Years:
    # Parameters
    X_V = [
        list(yearly_df.get_group(year)["其他成本"].values),
        list(yearly_df.get_group(year)["勞動成本"].values),
        list(yearly_df.get_group(year)["焚化處理量"].values)
    ]
    X_F = [
        list(yearly_df.get_group(year)["設計處理量"].values),
        list(yearly_df.get_group(year)["設計發電量"].values)
    ]
    Y = [list(yearly_df.get_group(year)["實際發電量"].values)]
    B = [list(yearly_df.get_group(year)["二氧化碳當量"].values)]
    yearly_prod_data[year] = prod_data(X_V, X_F, Y, B)

## 方法論
我們希望將假設不完全競爭市場的 decentralize model 套用在台灣的焚化爐上，並擴展到跨時間維度。在假設各個發電廠會最大化自己在所有時間區段的總盈餘情況下計算 Nash 均衡。

### 計算 $\Delta B_r^-$ 和 $\Delta B_r^+$
$$
\max \sum_{k\in K}\lambda_{kr}B_{k} \\
\text{s.t.} \sum_{k\in K}\lambda_{kr}X_{ik} \leq X_{ir}, \quad \forall i \in I_f \\
\sum_{k\in K}\lambda_{kr}X_{ik} \leq x_{ir}, \quad \forall i \in I_v \\
\sum_{k\in K}\lambda_{kr}Y_{k} \geq y_r \\
\lambda_{kr}, x_{ir}, y_r \geq 0, \forall k \in K, i \in I_v
$$

In [7]:
def initial_allocation(year, production_data): 
    X_V = production_data[year].x_v
    X_F = production_data[year].x_f
    Y = production_data[year].y
    B = production_data[year].b
    # Save CRS for each DMU
    crs = [0 for r in K]

    for r in K:
        md11 = Model('model')
        l = md11.addVars(K, vtype=GRB.CONTINUOUS, name='l')
        x = md11.addVars(I_V, vtype=GRB.CONTINUOUS, name='x')
        y = md11.addVars(J, vtype=GRB.CONTINUOUS, name='y')  

        md11.setObjective(
            quicksum(l[k] * B[0][k] for k in K), GRB.MAXIMIZE
        )

        md11.addConstrs(
            quicksum(l[k] * X_F[i][k] for k in K) <= X_F[i][r]
            for i in I_F
        )

        md11.addConstrs(
            quicksum(l[k] * X_V[i][k] for k in K) <= x[i]
            for i in I_V
        )

        md11.addConstrs(
            quicksum(l[k] * Y[j][k] for k in K) >= y[j]
            for j in J
        )

        md11.params.LogToConsole = 0
        # md11.params.NonConvex = 2
        md11.optimize() 

        print(df.loc[r,"焚化廠名稱"])
        for var in md11.getVars():
            print(f"{var.VarName}, {var.X}")

        # to load crs
        crs[r] = md11.objVal
    return crs

In [8]:
initial_permits = {}
year = 2020
initial_permits[year] = initial_allocation(year, yearly_prod_data)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-11-03
宜蘭縣利澤垃圾資源回收(焚化)廠
l[0], 1.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
l[13], 0.0
l[14], 0.0
l[15], 0.0
l[16], 0.0
l[17], 0.0
l[18], 0.0
l[19], 0.0
l[20], 0.0
l[21], 0.0
l[22], 0.0
l[23], 0.0
x[0], 960330.0
x[1], 1033323.0
x[2], 182263.4
y[0], 0.0
屏東縣崁頂垃圾資源回收(焚化)廠
l[0], 1.5
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
l[13], 0.0
l[14], 0.0
l[15], 0.0
l[16], 0.0
l[17], 0.0
l[18], 0.0
l[19], 0.0
l[20], 0.0
l[21], 0.0
l[22], 0.0
l[23], 0.0
x[0], 1440495.0
x[1], 1549984.5
x[2], 273395.1
y[0], 0.0
苗栗縣垃圾焚化廠
l[0], 0.7885483014861996
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
l[13], 0.08957006369426741
l[14], 0.0
l[15], 0.0
l[16], 0.0
l[17], 0.0
l[18], 0.0
l[19], 0.0
l[20

# 計算碳排放權的分配
### Decentralized model

In [9]:
def decentralized_allocation(production_data):
    X_V = production_data.x_v
    X_F = production_data.x_f
    Y = production_data.y
    B = production_data.b

    # Build Model
    md2 = Model('model')
    LB = -GRB.INFINITY
    _lambda = md2.addVars(K, K, vtype=GRB.CONTINUOUS, name='lambda')
    x = md2.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='x')
    y = md2.addVars(K, vtype=GRB.CONTINUOUS, name='y') 
    b = md2.addVars(K, vtype=GRB.CONTINUOUS, name='b', lb=LB) 
    phi1 = md2.addVars(I_F, K, vtype=GRB.CONTINUOUS, name='phi1', lb=LB)
    phi2 = md2.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='phi2', lb=LB)
    phi3 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='phi3', lb=LB)
    phi4 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='phi4', lb=LB)
    phi5 = md2.addVar(vtype=GRB.CONTINUOUS, name='phi5', lb=LB)
    phi6 = md2.addVar(vtype=GRB.CONTINUOUS, name='phi6', lb=LB)
    phi7 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='phi7', lb=LB)
    phi8 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='phi8', lb=LB)
    a1 = md2.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='a1')
    a2 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a2')
    a3 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a3')
    a4 = md2.addVars(K, K, vtype=GRB.CONTINUOUS, name='a4')
    a5 = md2.addVars(I_F, K, vtype=GRB.CONTINUOUS, name='a5')
    a6 = md2.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='a6')
    a7 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a7')
    a8 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a8')
    a9 = md2.addVar(vtype=GRB.CONTINUOUS, name='a9')
    a10 = md2.addVar(vtype=GRB.CONTINUOUS, name='a10')
    a11 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a11')
    a12 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a12')

    md2.setObjective(
        quicksum(a1[i, r] for i in I_V for r in K) 
        + quicksum(a2[r] for r in K)
        + quicksum(a3[r] for r in K)
        + quicksum(a4[k, r] for k in K for r in K)
        + quicksum(a5[i, r] for i in I_F for r in K)
        + quicksum(a6[i, r] for i in I_V for r in K)
        + quicksum(a7[r] for r in K)
        + quicksum(a8[r] for r in K)
        + a9 + a10
        + quicksum(a11[r] for r in K)
        + quicksum(a12[r] for r in K)
        , GRB.MINIMIZE
    )

    ## KKT 
    md2.addConstrs(
        (
            1 - (phi2[i,r]) 
        ) - a1[i, r] == 0
        for i in I_V for r in K
    )

    md2.addConstrs(
        (P_Y - tao * (
            quicksum(y[k] for k in K) + Y_bar   #Y_hat
        ) - tao * y[r] - phi3[r] + phi5) 
        - a2[r] == 0
        for r in K
    )

    md2.addConstrs(
        epsolon - phi4[r] + phi6 + phi7[r] - phi8[r] 
        - a3[r] == 0
        for r in K
    )

    md2.addConstrs(

        (
            - quicksum(phi1[i,r] * X_F[i][k] for i in I_F)  
            - quicksum(phi2[i,r] * X_V[i][k] for i in I_V)
            + phi3[r] * Y[0][k]
            - phi4[r] * B[0][k]
        ) - a4[k, r] == 0
        for r in K for k in K
    )

    md2.addConstrs(
        
        ( 
            quicksum(_lambda[k,r] * X_F[i][k] for k in K) - X_F[i][r] 
        ) - a5[i, r] == 0
        for i in I_F for r in K
    )

    md2.addConstrs(

        ( 
            quicksum(_lambda[k,r] * X_V[i][k] for k in K) - x[i,r] 
        ) - a6[i, r] == 0
        for i in I_V for r in K
    )

    md2.addConstrs(

        ( 
            y[r] - quicksum(_lambda[k,r] * Y[0][k] for k in K) 
        ) - a7[r] == 0
        for r in K
    )

    md2.addConstrs(
        quicksum(_lambda[k,r] * B[0][k] for k in K) - B[0][r] + b[r]
        - a8[r] == 0
        for r in K
    )

    md2.addConstr(

        (Demand - quicksum(y[r] for r in K) )
        - a9 == 0
    )

    md2.addConstr(
        Cap - quicksum(b[r] for r in K) - a10 == 0
    )

    md2.addConstrs(
        phi7[r] * 
        ( 
            B_minus[r] - b[r]
        ) - a11[r] == 0
        for r in K
    )

    md2.addConstrs(
        phi8[r] *
        ( 
            b[r] - B_plus[r]
        ) - a12[r] == 0
        for r in K
    )


    md2.write("decentralized_model.lp")
    # md2.params.LogToConsole = 0
    md2.params.NonConvex = 2
    md2.optimize() 

    # Return variables as objects
    result = prod_data(
        [[0 for _ in K] for _ in I_V],
        [[0 for _ in K] for _ in I_F],
        [[0 for _ in K] for _ in J],
        [[0 for _ in K] for _ in M]
    )
    x_sol = md2.getAttr('X',x)
    y_sol = md2.getAttr('X',y)
    b_sol = md2.getAttr('X',b)
    permits = [0 for _ in K]
    for r in K:
        # for i in I_V:
        #     result.x_v[i][r] = x_sol[(i,r)]
        # result.y[0][r] = y_sol[(r)]
        result.b[0][r] = B[0][r] - b_sol[(r)]
        permits[r] = b_sol[(r)]
    result.x_v = production_data.x_v
    result.x_f = production_data.x_f
    result.y = production_data.y
    revenue = md2.objVal

    for var in md2.getVars():
        print(f"{var.VarName}, {var.X}")

    # # Save variables as csv
    # csv_file = open(f"../results/permit_{year}.csv", "a")
    # for var in md2.getVars(): 
    #     writer = csv.writer(csv_file)
    #     writer.writerow([var.VarName, var.X])
    # csv_file.close()

    return result, permits, revenue


### Lozano model

In [10]:
def lozano_allocation(production_data): 
    X_V = production_data.x_v
    X_F = production_data.x_f
    Y = production_data.y
    B = production_data.b

    # Phase1: Maximize y
    md1 = Model('model')
    LB = -GRB.INFINITY
    _lambda = md1.addVars(K, K, vtype=GRB.CONTINUOUS, name='lambda')
    _gamma = md1.addVars(J, vtype=GRB.CONTINUOUS, name='gamma')
    x = md1.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='x')
    y = md1.addVars(J, K, vtype=GRB.CONTINUOUS, name='y') 
    b = md1.addVars(M, K, vtype=GRB.CONTINUOUS, name='b', lb=LB) 

    md1.setObjective(
        1/len(J) * quicksum(_gamma[j] for j in J), GRB.MAXIMIZE
    )
    for r in K:
        md1.addConstrs(
            quicksum(_lambda[k,r] * X_F[i][k] for k in K) <= X_F[i][r]
            for i in I_F
        )
        md1.addConstrs(
            quicksum(_lambda[k,r] * X_V[i][k] for k in K) <= x[i, r]
            for i in I_V
        )
        md1.addConstrs(
            quicksum(_lambda[k,r] * Y[j][k] for k in K) >= y[j, r]
            for j in J
        )
        md1.addConstrs(
            quicksum(_lambda[k,r] * B[m][k] for k in K) == b[m, r]
            for m in M
        )
        md1.addConstrs(
            x[i, r] <= X_V[i][r]
            for i in I_V
        )
        md1.addConstrs(
            y[j, r] >= Y[j][r]
            for j in J
        )
    md1.addConstrs(
        quicksum(y[j, r] for r in K) >= _gamma[j] * quicksum(Y[j][r] for r in K)
        for j in J
    )
    md1.addConstrs(
        quicksum(b[m, r] for r in K) <= quicksum(B[m][r] for r in K)
        for m in M
    )

    md1.params.LogToConsole = 0
    md1.optimize() 


    # Phase2: Minimize b
    y_sol = md1.getAttr('X',y)
    Y_star = [[y_sol[(0,r)] for r in K]]

    md2 = Model('model')
    LB = -GRB.INFINITY
    _lambda = md2.addVars(K, K, vtype=GRB.CONTINUOUS, name='lambda')
    _theta = md2.addVars(M, vtype=GRB.CONTINUOUS, name='theta', ub=1)
    x = md2.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='x')
    y = md2.addVars(J, K, vtype=GRB.CONTINUOUS, name='y') 
    b = md2.addVars(M, K, vtype=GRB.CONTINUOUS, name='b', lb=LB) 

    md2.setObjective(
        1/len(M) * quicksum(_theta[m] for m in M), GRB.MINIMIZE
    )
    for r in K:
        md2.addConstrs(
            quicksum(_lambda[k,r] * X_F[i][k] for k in K) <= X_F[i][r]
            for i in I_F
        )
        md2.addConstrs(
            quicksum(_lambda[k,r] * X_V[i][k] for k in K) <= x[i, r]
            for i in I_V
        )
        md2.addConstrs(
            quicksum(_lambda[k,r] * Y[j][k] for k in K) >= y[j, r]
            for j in J
        )
        md2.addConstrs(
            quicksum(_lambda[k,r] * B[m][k] for k in K) == b[m, r]
            for m in M
        )
        md2.addConstrs(
            x[i, r] <= X_V[i][r]
            for i in I_V
        )
        md2.addConstrs(
            y[j, r] >= Y[j][r]
            for j in J
        )

    md2.addConstrs(
        quicksum(y[j, k] for k in K) >= quicksum(Y_star[j][k] for k in K)
        for j in J
    )
    
    md2.addConstrs(
        quicksum(b[m, r] for r in K) <= _theta[m] * quicksum(B[m][r] for r in K)
        for m in M
    )
    # md2.addConstrs(
    #     quicksum(b[m, r] for r in K) == Cap  # 加一個限制，使得碳排放總量 == Cap 要求的量
    #     for m in M
    # )

    md2.params.LogToConsole = 0
    md2.optimize() 

    # Phase3: Minimize x
    theta_sol = md2.getAttr('X',_theta)
    theta_star = [theta_sol[(m)] for m in M]
    print("theta_star", theta_star)

    md3 = Model('model')
    LB = -GRB.INFINITY
    _lambda = md3.addVars(K, K, vtype=GRB.CONTINUOUS, name='lambda')
    _beta = md3.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='beta', ub=1)
    x = md3.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='x')
    y = md3.addVars(J, K, vtype=GRB.CONTINUOUS, name='y') 
    b = md3.addVars(M, K, vtype=GRB.CONTINUOUS, name='b', lb=LB) 

    md3.setObjective(
        1/len(I_V) * quicksum(_beta[i,k] for i in I_V for k in K), GRB.MINIMIZE
    )
    for r in K:
        md3.addConstrs(
            quicksum(_lambda[k,r] * X_F[i][k] for k in K) <= X_F[i][r]
            for i in I_F
        )
        md3.addConstrs(
            quicksum(_lambda[k,r] * X_V[i][k] for k in K) <= x[i, r]
            for i in I_V
        )
        md3.addConstrs(
            quicksum(_lambda[k,r] * Y[j][k] for k in K) >= y[j, r]
            for j in J
        )
        md3.addConstrs(
            quicksum(_lambda[k,r] * B[m][k] for k in K) == b[m, r]
            for m in M
        )
        md3.addConstrs(
            x[i, r] == _beta[i,r] * X_V[i][r]
            for i in I_V
        )
        md3.addConstrs(
            y[j, r] >= Y[j][r]
            for j in J
        )

    md3.addConstrs(
        quicksum(y[j, r] for r in K) >= quicksum(Y_star[j][r] for r in K)
        for j in J
    )

    md3.params.LogToConsole = 0
    md3.optimize() 

    """Return variables as objects"""
    result = prod_data(
        [[0 for _ in K] for _ in I_V],
        [[0 for _ in K] for _ in I_F],
        [[0 for _ in K] for _ in J],
        [[0 for _ in K] for _ in M]
    )
    x_sol = md3.getAttr('X',x)
    y_sol = md3.getAttr('X',y)
    b_sol = md3.getAttr('X',b)
    permits = [0 for _ in K]
    for r in K:
        # for i in I_V:
        #     result.x_v[i][r] = x_sol[(i,r)]
        # for j in J:
        #     result.y[0][r] = y_sol[(j,r)]
        for m in M:
            result.b[0][r] = b_sol[(m,r)]
            permits[r] = production_data.b[0][r] - b_sol[(m,r)]
    result.x_v = production_data.x_v
    result.x_f = production_data.x_f
    result.y = production_data.y
    revenue = md3.objVal
    
    for var in md1.getVars():
        print(f"{var.VarName}, {var.X}")

    return result, permits, revenue

## Feng Model

In [25]:
def feng_allocation(production_data): 
    X_V = production_data.x_v
    X_F = production_data.x_f
    Y = production_data.y
    B = production_data.b

    md1 = Model('model')
    LB = -GRB.INFINITY
    _lambda = md1.addVars(K, K, vtype=GRB.CONTINUOUS, name='lambda')
    b = md1.addVars(M, K, vtype=GRB.CONTINUOUS, name='b', lb=LB) 
    w = md1.addVars(K, vtype=GRB.CONTINUOUS, name='w', lb=0.000000000001, ub=1)

    md1.setObjective(
        quicksum(
            _lambda[k, r] * Y[j][k] for j in J for k in K for r in K
        ), GRB.MAXIMIZE
    )
    for r in K:
        md1.addConstrs(
            quicksum(_lambda[k,r] * X_F[i][k] for k in K) <= w[r] * X_F[i][r]
            for i in I_F
        )
        md1.addConstrs(
            quicksum(_lambda[k,r] * X_V[i][k] for k in K) <= w[r] * X_V[i][r]
            for i in I_V
        )
        md1.addConstrs(
            quicksum(_lambda[k,r] * B[m][k] for k in K) == B[m][r] - b[m, r]
            for m in M
        )
        md1.addConstr(
            b[0, r] >= B_minus[r]
        )
        md1.addConstr(
            b[0, r] <= B_plus[r]
        )
        md1.addConstr(
            quicksum(_lambda[k, r] for k in K) == w[r]
        )
    md1.addConstr(
        quicksum(b[0, k] for k in K) == Cap
    )


    # md1.params.LogToConsole = 0
    md1.optimize() 

    """Return variables as objects"""
    result = prod_data(
        [[0 for _ in K] for _ in I_V],
        [[0 for _ in K] for _ in I_F],
        [[0 for _ in K] for _ in J],
        [[0 for _ in K] for _ in M]
    )
    # x_sol = md1.getAttr('X',x)
    # y_sol = md1.getAttr('X',y)
    b_sol = md1.getAttr('X',b)
    permits = [0 for _ in K]
    for r in K:
        # for i in I_V:
        #     result.x_v[i][r] = x_sol[(i,r)]
        # for j in J:
        #     result.y[0][r] = y_sol[(j,r)]
        for m in M:
            result.b[m][r] = B[m][r] - b_sol[(m,r)]
            permits[r] = b_sol[(m,r)]
    result.x_v = production_data.x_v
    result.x_f = production_data.x_f
    result.y = production_data.y
    total_output = md1.objVal

    for var in md1.getVars():
        print(f"{var.VarName}, {var.X}")

    return result, permits, total_output

In [41]:
B_minus

[0.0,
 -16420.725000000006,
 -2089.167043192676,
 -18187.01250000001,
 -22781.73749999999,
 -62021.13750000001,
 -64254.600000000006,
 -127296.37499999994,
 -3292.4249999999884,
 -23192.962499999965,
 -22251.809937300917,
 -73787.31781200238,
 -40116.225000000006,
 0.0,
 -3893.1749999999593,
 -14428.79999999996,
 -30437.848313097114,
 -8454.524999999994,
 -1191.0,
 -21121.21341358556,
 -82800.53889828819,
 -142689.14999999997,
 -16651.79999999996,
 -49078.82963276272]

## Fare Distance Model: to evaluate efficiency of the current permit allocation method.
x, y 是原本給定的參數，只有 b 為分配後的結果

In [26]:
def fare_efficiency(production_data): 
    X_V = production_data.x_v
    X_F = production_data.x_f
    Y = production_data.y
    B = production_data.b
    # Save CRS for each DMU

    efficiency = [0 for r in K]

    for r in K:
        md11 = Model('model')
        l = md11.addVars(K, vtype=GRB.CONTINUOUS, name='l')
        _theta = md11.addVar(vtype=GRB.CONTINUOUS, name='theta')
        _epsilon = md11.addVar(vtype=GRB.CONTINUOUS, name='epsilon', ub=1)

        md11.setObjective(
            _theta, GRB.MINIMIZE
        )

        md11.addConstrs(
            quicksum(l[k] * X_F[i][k] for k in K) <= X_F[i][r]
            for i in I_F
        )

        md11.addConstrs(
            quicksum(l[k] * X_V[i][k] for k in K) <= _epsilon * X_V[i][r]
            for i in I_V
        )

        md11.addConstrs(
            quicksum(l[k] * Y[j][k] for k in K) >= Y[j][r]
            for j in J
        )


        md11.addConstrs(
            quicksum(l[k] * B[m][k] for k in K) == _theta * B[m][r]
            for m in M
        )

        md11.addConstr(
            quicksum(l[k] for k in K) == _epsilon
        )

        md11.params.LogToConsole = 0
        md11.optimize() 

        print(list(yearly_df.get_group(2020)["焚化廠名稱"].values)[r])
        for var in md11.getVars():
            print(f"{var.VarName}, {var.X}")

        # print("sum Y :" ,quicksum(l[k] * Y[0][k] for k in K).getValue())
        # print("Y[j][r] :" ,quicksum(Y[0][r]))

        # to load crs
        efficiency[r] = md11.objVal
    return efficiency

## 開始分配碳權

In [42]:
year = 2020    

B_minus = (np.array(B[0])- np.array(initial_permits[year])).tolist()  #減碳下限
# B_minus = [0 for k in K]
B_plus = B[0]   #減碳上限為原本的排放量
Cap = sum(B[0]) * 0.05    #Cap of trade
Demand = 2000000  #Demand of electricity
# P_X = 590
P_Y = 4 * 1e7
tao = 123
Y_bar = 10975
epsolon = 0.001

print("FinalY:", yearly_prod_data[2020].y)
print("lozano")
lozano_prod, lozano_permits, _  = lozano_allocation(yearly_prod_data[2020])
print("FinalY:", yearly_prod_data[2020].y)
print("feng")
feng_prod, feng_permits, _  = feng_allocation(yearly_prod_data[2020])
print("FinalY:", yearly_prod_data[2020].y)
print("nash crs")
decentralized_prod, decentralized_permits, revenue  = decentralized_allocation(yearly_prod_data[2020])
print("FinalY:", yearly_prod_data[2020].y)


FinalY: [[99631.87, 132776.5, 84818.29, 235050.3, 223770.7, 215609.2, 54236.19, 212407.2, 114549.6, 134504.3, 102684.9, 123818.5, 134164.5, 16811.61, 157251.3, 131201.5, 72399.38, 160709.0, 168598.7, 30631.83, 79307.77, 179905.0, 149915.3, 73605.6]]
lozano
theta_star [0.9597513308428073]
lambda[0,0], 0.0
lambda[0,1], 0.0
lambda[0,2], 0.0
lambda[0,3], 0.0
lambda[0,4], 0.0
lambda[0,5], 0.0
lambda[0,6], 0.0
lambda[0,7], 0.0
lambda[0,8], 0.0
lambda[0,9], 0.0
lambda[0,10], 0.0
lambda[0,11], 0.0
lambda[0,12], 0.0
lambda[0,13], 0.0
lambda[0,14], 0.0
lambda[0,15], 0.0
lambda[0,16], 0.0
lambda[0,17], 0.0
lambda[0,18], 0.0
lambda[0,19], 0.0
lambda[0,20], 0.0
lambda[0,21], 0.0
lambda[0,22], 0.0
lambda[0,23], 0.0
lambda[1,0], 0.0
lambda[1,1], 0.0
lambda[1,2], 0.0
lambda[1,3], 0.0
lambda[1,4], 0.0
lambda[1,5], 0.0
lambda[1,6], 0.0
lambda[1,7], 0.0
lambda[1,8], 0.0
lambda[1,9], 0.0
lambda[1,10], 0.0
lambda[1,11], 0.0
lambda[1,12], 0.0
lambda[1,13], 0.0
lambda[1,14], 0.0
lambda[1,15], 0.0
lambda[1,16

In [44]:
permits_table = pd.DataFrame([lozano_permits, feng_permits, decentralized_permits], index=["Lozano", "Feng", "Nash CRS"]).T
permits_table = permits_table.round(1)
permits_table

,Lozano,Feng,Nash CRS
0,12069.9,9551.7,10021.8
1,0.0,0.0,53921.0
2,3514.0,0.0,-323.6
3,0.0,0.0,43467.0
4,0.0,0.0,77151.0
5,0.0,0.0,24142.0
6,0.0,0.0,0.0
7,0.0,37247.9,5270.9
8,0.0,0.0,7552.7
9,0.0,22588.1,78219.0


In [45]:
permits_table.sum()

Lozano      177955.9
Feng        221070.6
Nash CRS    221070.6
dtype: float64

In [46]:
permits_table.describe()

,Lozano,Feng,Nash CRS
count,24.000000,24.00000,24.000000
mean,7414.829167,9211.27500,9211.275000
std,14675.482174,22207.57006,41008.812848
min,-0.000000,0.00000,-82383.800000
25%,0.000000,0.00000,-1739.350000
50%,0.000000,0.00000,2890.950000
75%,5652.975000,4671.82500,31679.325000
max,48484.300000,101225.10000,78219.000000


In [31]:
pd.concat([permits_table, permits_table.describe()]).to_csv( "../results/permits_table.csv")


## 計算效率

In [32]:
# original_efficiency = {}
# decentralized_efficiency = {}
""" 計算效率 """
print("FinalY:", yearly_prod_data[2020].y)
original_efficiency = fare_efficiency(yearly_prod_data[2020])
print("FinalY:", yearly_prod_data[2020].y)

print("Nash CRS")
decentralized_efficiency = fare_efficiency(decentralized_prod)
print("FinalY:", yearly_prod_data[2020].y)

print("lozano")
lozano_efficiency = fare_efficiency(lozano_prod)
print("FinalY:", yearly_prod_data[2020].y)

print("feng")
feng_efficiency = fare_efficiency(feng_prod)
print("FinalY:", yearly_prod_data[2020].y)


FinalY: [[99631.87, 132776.5, 84818.29, 235050.3, 223770.7, 215609.2, 54236.19, 212407.2, 114549.6, 134504.3, 102684.9, 123818.5, 134164.5, 16811.61, 157251.3, 131201.5, 72399.38, 160709.0, 168598.7, 30631.83, 79307.77, 179905.0, 149915.3, 73605.6]]
宜蘭縣利澤垃圾資源回收(焚化)廠
l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.09397537252943561
l[6], 0.0
l[7], 0.0
l[8], 0.5987269884253072
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
l[13], 0.13185562095818823
l[14], 0.05449413009651386
l[15], 0.0
l[16], 0.0
l[17], 0.0
l[18], 0.0
l[19], 0.0
l[20], 0.0
l[21], 0.0
l[22], 0.0
l[23], 0.0
theta, 0.879052112009445
epsilon, 0.879052112009445
屏東縣崁頂垃圾資源回收(焚化)廠
l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.39790373481040753
l[6], 0.0
l[7], 0.0
l[8], 0.41016986580936016
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
l[13], 0.0
l[14], 0.0
l[15], 0.0
l[16], 0.0
l[17], 0.0
l[18], 0.0
l[19], 0.0
l[20], 0.0
l[21], 0.0
l[22], 0.0
l[23], 0.0
theta, 0.8080736006197676
epsilon, 0.8080736006197677
苗栗縣垃圾焚化

In [33]:
yearly_prod_data[2020].y

[[99631.87,
  132776.5,
  84818.29,
  235050.3,
  223770.7,
  215609.2,
  54236.19,
  212407.2,
  114549.6,
  134504.3,
  102684.9,
  123818.5,
  134164.5,
  16811.61,
  157251.3,
  131201.5,
  72399.38,
  160709.0,
  168598.7,
  30631.83,
  79307.77,
  179905.0,
  149915.3,
  73605.6]]

In [34]:
efficiency_table = pd.DataFrame([original_efficiency, lozano_efficiency, feng_efficiency, decentralized_efficiency], index=["Original","Lozano", "Feng", "Nash CRS"]).T

In [35]:
efficiency_table

,Original,Lozano,Feng,Nash CRS
0,0.879052,0.927863,0.945090,0.811918
1,0.808074,0.805851,0.776351,0.818323
2,1.000000,1.000000,1.000000,1.000000
3,1.000000,1.000000,1.000000,1.000000
4,1.000000,1.000000,1.000000,1.000000
5,1.000000,1.000000,1.000000,1.000000
6,0.450307,0.442097,0.450307,0.376615
7,0.855392,0.855392,0.985149,0.704806
8,1.000000,1.000000,1.000000,1.000000
9,0.538657,0.538657,0.526706,0.522058


In [36]:
efficiency_table.describe()

,Original,Lozano,Feng,Nash CRS
count,24.000000,24.000000,24.000000,24.000000
mean,0.878672,0.887811,0.834794,0.789367
std,0.164568,0.160585,0.253182,0.241324
min,0.450307,0.442097,0.000000,0.376615
25%,0.788894,0.801134,0.730992,0.560083
50%,0.987106,0.987106,0.977854,0.909161
75%,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000


In [37]:
pd.concat([efficiency_table, efficiency_table.describe()]).to_csv( "../results/efficiency_table.csv")

## Electricity generation

In [38]:
print(sum(Y[0]),sum(lozano_prod.y[0]),sum(feng_prod.y[0]),sum(decentralized_prod.y[0]))

3088359.04 3088359.04 3088359.04 3088359.04
